In [2]:
%run "../Functions/4. User comparison.ipynb"

rmdfTestUsers read_csv success (1/3)
rmdf1522 read_csv success (2/3)
rmdf160 read_csv success (3/3)
gform read_csv success
gformFR read_csv success
temporalities set (user answer method)


IntProgress(value=0, max=1708)

IntProgress(value=0, max=854)

profile info set


# Prepare data

## Prepare Google form data

In [3]:
# Rename columns of the Google Forms table with tags independantly of form language
#columnTags = ["timestamp", "gameInterest", "gameFrequency", "age", "gender", "biologyStudy", "biologyInterest", "synthBioKnowledge", "biobrickKnowledge", "previousVersion", "previousPlay", "arcadePlay", "androidPlay", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27", "comments", "anonymousID", "lang", "temporality"]
columnTagsDictionary = {
    QTimestamp : 'timestamp',
    QCuriosityBiology : 'QCuriosityBiology',
    QCuriositySyntheticBiology : 'QCuriositySyntheticBiology',
    QCuriosityVideoGames : 'QCuriosityVideoGames',
    QCuriosityEngineering : 'QCuriosityEngineering',
    QPlayed : 'previousPlay',
    QAge : 'age',
    QGender : 'gender',
    QInterestVideoGames : 'gameInterest',
    QInterestBiology : 'biologyInterest',
    QStudiedBiology : 'biologyStudy',
    QPlayVideoGames : 'gameFrequency',
    QHeardSynBioOrBioBricks : 'synthBioKnowledge',
    QVolunteer : 'QVolunteer',
    QEnjoyed : 'QEnjoyed',
    QGenotypePhenotype : 'QGenotypePhenotype',
    QBioBricksDevicesComposition : 'QBioBricksDevicesComposition',
    QAmpicillin : 'QAmpicillin',
    QBBNamePlasmid : 'QBBNamePlasmid',
    QBBFunctionTER : 'QBBFunctionTER',
    QBBNamePromoter : 'QBBNamePromoter',
    QBBFunctionGameCDS : 'QBBFunctionGameCDS',
    QBBNameTerminator : 'QBBNameTerminator',
    QBBFunctionBiologyCDS : 'QBBFunctionBiologyCDS',
    QBBNameRBS : 'QBBNameRBS',
    QBBExampleCDS : 'QBBExampleCDS',
    QBBNameCDS : 'QBBNameCDS',
    QBBFunctionPR : 'QBBFunctionPR',
    QBBFunctionRBS : 'QBBFunctionRBS',
    QBBFunctionPLASMID : 'QBBFunctionPLASMID',
    QBBNameOperator : 'QBBNameOperator',
    QDeviceRbsPconsFlhdcTer : 'QDeviceRbsPconsFlhdcTer',
    QDevicePconsRbsFlhdcTer : 'QDevicePconsRbsFlhdcTer',
    QDevicePbadRbsGfpTer : 'QDevicePbadRbsGfpTer',
    QDevicePbadGfpTbsTer : 'QDevicePbadGfpTbsTer',
    QDeviceGfpRbsPconsTer : 'QDeviceGfpRbsPconsTer',
    QDevicePconsGfpRbsTer : 'QDevicePconsGfpRbsTer',
    QDeviceAmprRbsPconsTer : 'QDeviceAmprRbsPconsTer',
    QDeviceRbsPconsAmprTer : 'QDeviceRbsPconsAmprTer',
    QGreenFluorescence : 'QGreenFluorescence',
    QUnequipDevice : 'QUnequipDevice',
    QDevicePbadRbsAraTer : 'QDevicePbadRbsAraTer',
    QRemarks : 'comments',
    QUserId : 'anonymousID',
    QLanguage : 'lang',
    QTemporality : 'temporality',
}
columnQuestions = gform.columns.values.tolist()
googleData = gform.rename(columns=columnTagsDictionary)
#googleData.head()

In [16]:
# Replaces answers to scientific questions in the questionnaires by their values (True or False)
correctedData = googleData.copy()
scientificQuestions = correctAnswers[correctAnswers.apply(len) != 0].index

for rowId in range(correctedData.shape[0]):
    # Get the correction for each subject
    playerId = correctedData.loc[rowId, "anonymousID"]
    correction = getCorrections(playerId)
    if correction.shape[1] > 0:
        # If subject has answered questionnaire
        # Replace scientific answers by their correction
        for question in scientificQuestions:
            correctedData.loc[rowId, question] = int(correction.loc[question, "corrections" + str(rowId)])
    
correctedData = correctedData.rename(columns=columnTagsDictionary)
#correctedData.tail(15)

### For association rule mining

In [12]:
# Get only answers to scientific questions
correctedScientific = correctedData.loc[:, scientificQuestions]
#correctedScientific.head()

In [13]:
correctedScientific

,Genotype and phenotype,BioBricks and devices composition,Ampicillin antibiotic,Name: Plasmid,Function: TER,Name: PR,Function - game: CDS,Name: TER,Function - biology: CDS,Name: RBS,...,Device: PCONS:RBS:FLHDC:TER,Device: PBAD:RBS:GFP:TER,Device: PBAD:GFP:RBS:TER XXX,Device: GFP:RBS:PCONS:TER XXX,Device: PCONS:GFP:RBS:TER XXX,Device: AMPR:RBS:PCONS:TER XXX,Device: RBS:PCONS:AMPR:TER XXX,Green fluorescence,Unequip the movement device: effect,Device: PBAD:RBS:ARA:TER
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### For clustering

#### With full answers

In [ ]:
# Remove timestamp and comments features
codedData = googleData.copy().drop(['timestamp', "comments"], axis=1)
codedData.head()

In [ ]:
# Code answers with integers when possible

# Define equivalences
# gameInterest
gameInterestCoding = {"A lot": 4, "Beaucoup": 4, "Enormément": 5, "Extremely": 5, "Moderately": 3, "Moyennement": 3, "Slightly": 2, "Un peu": 2, "I don't know": 3, "Je ne sais pas": 3, "Not at all": 1, "Pas du tout": 1}
# gameFrequency
gameFrequencyCoding = {"A lot": 4, "Beaucoup": 4, "Enormément": 5, "Extremely": 5, "Moderately": 3, "Moyennement": 3, "Rarely": 2, "Un peu": 2, "I don't know": 3, "Je ne sais pas": 3, "Not at all": 1, "Pas du tout": 1}
# biologyStudy
biologyStudyCoding = {"Not even in middle school": 0, "Jamais": 0, "Jamais, pas même au collège": 0, "Until the end of middle school": 1, "Jusqu'au brevet": 1, "Until the end of high school": 2, "Jusqu'au bac": 2, "Until bachelor's degree": 3, "Jusqu'à la license": 3, "At least until master's degree": 4, "Au moins jusqu'au master": 4, "I don't know": 0, "Je ne sais pas": 0}
# biologyInterest
biologyInterestCoding = {"A lot": 4, "Beaucoup": 4, "Enormément": 5, "Extremely": 5, "Moderately": 3, "Moyennement": 3, "Slightly": 2, "Un peu": 2, "I don't know": 3, "Je ne sais pas": 3, "Not at all": 1, "Pas du tout": 1}
# synthBioKnowledge
# biobrickKnowledge
previousKnowledgeCoding = {"Yes": 1, "No": 0, "I don't know": 0, "Oui": 1, 'Non': 0, "Je ne sais pas": 0}
# previousVersion
# previousPlay
# arcadePlay
# androidPlay
previousPlayCoding = {"Multiple times": 3, "A few times": 2, "Once": 1, "Yes": 1, "No": 0, "I don't know": 0, "De nombreuses fois": 3, "Quelques fois": 2, "Une fois": 1, "Oui": 1, "Non": 0, "Je ne sais pas": 0}
# lang
languageCoding = {"en": 0, "fr": 1}
# temporality
temporalityCoding = {"before": 0, "after": 1, "undefined": -5}

# Fill NaN cells
codedData["biobrickKnowledge"].fillna("I don't know", inplace = True)
codedData["arcadePlay"].fillna("I don't know", inplace = True)
codedData["androidPlay"].fillna("I don't know", inplace = True)
codedData["previousPlay"].fillna("I don't know", inplace = True)

# Replace by code
for rowId in range(codedData.shape[0]):
    codedData.loc[rowId, "gameInterest"] = gameInterestCoding[codedData.loc[rowId, "gameInterest"]]
    codedData.loc[rowId, "gameFrequency"] = gameFrequencyCoding[codedData.loc[rowId, "gameFrequency"]]
    codedData.loc[rowId, "biologyStudy"] = biologyStudyCoding[codedData.loc[rowId, "biologyStudy"]]
    codedData.loc[rowId, "biologyInterest"] = biologyInterestCoding[codedData.loc[rowId, "biologyInterest"]]
    codedData.loc[rowId, "synthBioKnowledge"] = previousKnowledgeCoding[codedData.loc[rowId, "synthBioKnowledge"]]
    codedData.loc[rowId, "biobrickKnowledge"] = previousKnowledgeCoding[codedData.loc[rowId, "biobrickKnowledge"]]
    codedData.loc[rowId, "previousVersion"] = previousPlayCoding[codedData.loc[rowId, "previousVersion"]]
    codedData.loc[rowId, "previousPlay"] = previousPlayCoding[codedData.loc[rowId, "previousPlay"]]
    codedData.loc[rowId, "arcadePlay"] = previousPlayCoding[codedData.loc[rowId, "arcadePlay"]]
    codedData.loc[rowId, "androidPlay"] = previousPlayCoding[codedData.loc[rowId, "androidPlay"]]
    codedData.loc[rowId, "lang"] = languageCoding[codedData.loc[rowId, "lang"]]
    codedData.loc[rowId, "temporality"] = temporalityCoding[codedData.loc[rowId, "temporality"]]
    

In [ ]:
# One-Hot version
codedData = pd.get_dummies(codedData, prefix = ["gender", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"], columns = ["gender", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"])
#codedData.head()

In [ ]:
# Split the forms according to temporality
beforeForms = codedData.copy().loc[codedData["temporality"] == 0,:]
afterForms = codedData.copy().loc[codedData["temporality"] == 1,:]
undefForms = codedData.copy().loc[codedData["temporality"] == -5,:]
defForms = codedData.copy().loc[codedData["temporality"] >= 0,:] # Either before or after
# For subjects with both before and after forms, join the two
beforeAndAfterForms = pd.merge(beforeForms, afterForms, on="anonymousID", suffixes=('_before', '_after'))
#beforeForms.head()
#afterForms.head()
#undefForms.head()
#defForms.head()
#beforeAndAfterForms.head()

In [ ]:
# Remove ID feature
allForms = codedData.copy().drop("anonymousID", axis=1)
beforeForms.drop("anonymousID", axis=1, inplace = True)
afterForms.drop("anonymousID", axis=1, inplace = True)
undefForms.drop("anonymousID", axis=1, inplace = True)
defForms.drop("anonymousID", axis=1, inplace = True)
beforeAndAfterForms.drop("anonymousID", axis=1, inplace = True)

#### With corrected scientific answers

In [ ]:
# Remove timestamp and comments features
codedCorrectedData = correctedData.copy().drop(['timestamp', "comments"], axis=1)

In [ ]:
# Fill NaN cells
codedCorrectedData["biobrickKnowledge"].fillna("I don't know", inplace = True)
codedCorrectedData["arcadePlay"].fillna("I don't know", inplace = True)
codedCorrectedData["androidPlay"].fillna("I don't know", inplace = True)
codedCorrectedData["previousPlay"].fillna("I don't know", inplace = True)

# Replace by code
for rowId in range(codedData.shape[0]):
    codedCorrectedData.loc[rowId, "gameInterest"] = gameInterestCoding[codedCorrectedData.loc[rowId, "gameInterest"]]
    codedCorrectedData.loc[rowId, "gameFrequency"] = gameFrequencyCoding[codedCorrectedData.loc[rowId, "gameFrequency"]]
    codedCorrectedData.loc[rowId, "biologyStudy"] = biologyStudyCoding[codedCorrectedData.loc[rowId, "biologyStudy"]]
    codedCorrectedData.loc[rowId, "biologyInterest"] = biologyInterestCoding[codedCorrectedData.loc[rowId, "biologyInterest"]]
    codedCorrectedData.loc[rowId, "synthBioKnowledge"] = previousKnowledgeCoding[codedCorrectedData.loc[rowId, "synthBioKnowledge"]]
    codedCorrectedData.loc[rowId, "biobrickKnowledge"] = previousKnowledgeCoding[codedCorrectedData.loc[rowId, "biobrickKnowledge"]]
    codedCorrectedData.loc[rowId, "previousVersion"] = previousPlayCoding[codedCorrectedData.loc[rowId, "previousVersion"]]
    codedCorrectedData.loc[rowId, "previousPlay"] = previousPlayCoding[codedCorrectedData.loc[rowId, "previousPlay"]]
    codedCorrectedData.loc[rowId, "arcadePlay"] = previousPlayCoding[codedCorrectedData.loc[rowId, "arcadePlay"]]
    codedCorrectedData.loc[rowId, "androidPlay"] = previousPlayCoding[codedCorrectedData.loc[rowId, "androidPlay"]]
    codedCorrectedData.loc[rowId, "lang"] = languageCoding[codedCorrectedData.loc[rowId, "lang"]]
    codedCorrectedData.loc[rowId, "temporality"] = temporalityCoding[codedCorrectedData.loc[rowId, "temporality"]]

In [ ]:
# One-Hot versions of the above dataframes
codedCorrectedData = pd.get_dummies(codedCorrectedData, prefix = ["gender"], columns = ["gender"])
codedCorrectedData.head()

In [ ]:
# Split the forms according to temporality
beforeCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == 0,:]
afterCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == 1,:]
undefCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == -5,:]
defCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] >= 0,:] # Either before or after
# For subjects with both before and after forms, join the two
beforeAndAfterCorrectedForms = pd.merge(beforeCorrectedForms, afterCorrectedForms, on="anonymousID", suffixes=('_before', '_after'))
#beforeCorrectedForms.head()
#afterCorrectedForms.head()
#undefCorrectedForms.head()
#defCorrectedForms.head()
#beforeAndAfterCorrectedForms.head()

In [ ]:
# Remove ID feature
allCorrectedForms = codedCorrectedData.copy().drop("anonymousID", axis=1)
beforeCorrectedForms.drop("anonymousID", axis=1, inplace = True)
afterCorrectedForms.drop("anonymousID", axis=1, inplace = True)
undefCorrectedForms.drop("anonymousID", axis=1, inplace = True)
defCorrectedForms.drop("anonymousID", axis=1, inplace = True)
beforeAndAfterCorrectedForms.drop("anonymousID", axis=1, inplace = True)

### For Classification

In [ ]:
# Use defForms and defCorrectedForms for coded data

## RedMetrics

### For clustering

In [ ]:
# Fetch RedMetrics data for subjects which answered the gform
allData = getAllUserVectorData(getAllResponders(), _source=[])

In [ ]:
# Put subjects as rows and features as columns
allData = allData.transpose()
allData.head()

In [ ]:
# Drop useless columns in which all rows have the same value
allData.drop(["switch", "gotomooc"], axis=1, inplace = True)
# Fill NaN with negatives
allData.fillna(-1, inplace = True)

In [ ]:
# Get the subset of subjects who have answered before and after
fullProcessData = allData[allData["scorebefore"] >= 0]
fullProcessData = fullProcessData[fullProcessData["scoreafter"] >= 0]
#fullProcessData.loc["01e85778-2903-447b-bbab-dd750564ee2d",:]

#### Combined with questionnaire answers

In [ ]:
#Get the subset of subjects who have answered the after questionnaire
withAfterData = allData.copy()[allData["scoreafter"] >= 0]
withAfterData['anonymousID'] = withAfterData.index
withAfterData['anonymousID'] = withAfterData['anonymousID'].apply(lambda x: x.strip('"'))
# Join with questionnaire data with complete answers
afterNotCorrected = codedData.copy().loc[codedData["temporality"] == 1,:]
gameAndAfterData = pd.merge(withAfterData, afterNotCorrected, on="anonymousID")
gameAndAfterData.drop("anonymousID", axis=1, inplace = True)
#Join with questionnaire data with corrected answers
afterCorrected = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == 1,:]
gameAndCorrectedAfterData = pd.merge(withAfterData, afterCorrected, on="anonymousID")
gameAndCorrectedAfterData.drop("anonymousID", axis=1, inplace = True)
#gameAndAfterData.head()
#gameAndCorrectedAfterData.head()

### For classification

In [ ]:
allDataClassif = allData.copy()
allDataClassif['anonymousID'] = allData.index
allDataClassif['anonymousID'] = allDataClassif['anonymousID'].apply(lambda x: x.strip('"'))

# If checkpoint not reached, set time to 3600 (1h)
def floorCheckpoints(value):
    if value > 3600:
        return 3600
    return value
for col in allDataClassif.columns.values.tolist():
    if isinstance(col, np.int64):
        allDataClassif[col] = allDataClassif[col].apply(floorCheckpoints)
allDataClassif["completionTime"] = allDataClassif["completionTime"].apply(lambda x: min(x, 7200))        

# floor thoroughness
allDataClassif["thoroughness"]  = allDataClassif["thoroughness"].apply(lambda x: min(x, 1000))

#### Combined with questionnaire answers

In [ ]:
#Join with questionnaire data with corrected answers
gameAndCorrectedAfterDataClassif = pd.merge(withAfterData, afterCorrected, on="anonymousID")
gameAndCorrectedAfterDataClassif['anonymousID'] = gameAndCorrectedAfterDataClassif['anonymousID'].apply(lambda x: x.strip('"'))
gameAndCorrectedAfterDataClassif.drop(["scorebefore", "scoreundefined", "temporality"], axis=1, inplace = True)

for col in gameAndCorrectedAfterDataClassif.columns.values.tolist():
    if isinstance(col, np.int64):
        gameAndCorrectedAfterDataClassif[col] = gameAndCorrectedAfterDataClassif[col].apply(floorCheckpoints)
gameAndCorrectedAfterDataClassif["completionTime"] = gameAndCorrectedAfterDataClassif["completionTime"].apply(lambda x: min(x, 7200))

# floor thoroughness
gameAndCorrectedAfterDataClassif["thoroughness"]  = gameAndCorrectedAfterDataClassif["thoroughness"].apply(lambda x: min(x, 1000))

In [ ]:
#Get the subset of subjects who have answered the before questionnaire
withBeforeData = allData.copy()[allData["scorebefore"] >= 0]
withBeforeData['anonymousID'] = withBeforeData.index
withBeforeData['anonymousID'] = withBeforeData['anonymousID'].apply(lambda x: x.strip('"'))
beforeCorrected = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == 0,:]
#Join with questionnaire data with corrected answers
gameAndCorrectedBeforeDataClassif = pd.merge(withBeforeData, beforeCorrected, on="anonymousID")
gameAndCorrectedBeforeDataClassif['anonymousID'] = gameAndCorrectedBeforeDataClassif['anonymousID'].apply(lambda x: x.strip('"'))
gameAndCorrectedBeforeDataClassif.drop(["scoreafter", "scoreundefined", "temporality"], axis=1, inplace = True)

# If checkpoint not reached, set time to 3600 (1h)
for col in gameAndCorrectedBeforeDataClassif.columns.values.tolist():
    if isinstance(col, np.int64):
        gameAndCorrectedBeforeDataClassif[col] = gameAndCorrectedBeforeDataClassif[col].apply(floorCheckpoints)
gameAndCorrectedBeforeDataClassif["completionTime"] = gameAndCorrectedBeforeDataClassif["completionTime"].apply(lambda x: min(x, 7200))

# floor thoroughness
gameAndCorrectedBeforeDataClassif["thoroughness"]  = gameAndCorrectedBeforeDataClassif["thoroughness"].apply(lambda x: min(x, 1000))